In [3]:
'''
Moshe's idea:
1. Start with a large Boolean formula f (say, industrial SAT benchmark).
2. Generate many samples satisfying f or \neg f.
(Using, for example, [https://www.nature.com/articles/s42256-018-0002-3](https://www.nature.com/articles/s42256-018-0002-3))
3. Generate many samples satisfying f or \neg f.
(Using, for example, [https://www.nature.com/articles/s42256-018-0002-3](https://www.nature.com/articles/s42256-018-0002-3))
4. Train a DNN on this labeled sample.
5. Check whether it is a good apprcximation of f.
----------
To use this notebook, please install pysat: https://pysathq.github.io/installation.html
Perhaps the simplest way is with the following command: pip install python-sat[pblib,aiger]
(pysat might not work on Windows)
'''


# This is just a warm-up cell, to see how pysat works

# some examples of pysat usage at: https://github.com/pysathq/pysat/blob/master/examples/usage.py
from pysat.solvers import Glucose3
g = Glucose3()
g.add_clause([-1, 2])
g.add_clause([-2, 3])


for m in g.enum_models():  # implemented as a generator
    print(m)
g.delete()

[-1, -2, -3]
[-1, -2, 3]
[-1, 2, 3]
[1, 2, 3]


In [32]:
# to use this cell, extract bw_large.d.cnf from instances/blocksworld.tar.gz
# blocksworld is from satlib at https://www.cs.ubc.ca/~hoos/SATLIB/Benchmarks/SAT/PLANNING/BlocksWorld/blocksworld.tar.gz
# satlib website: https://www.cs.ubc.ca/~hoos/SATLIB/benchm.html

from pysat.formula import CNF

formula = CNF(from_file='instances/bw_large.d.cnf')

num_vars = formula.nv
sat_list = []
unsat_list = []

with Glucose3(bootstrap_with=formula) as solver:
    
    # for each positive (sat) instance, flip a literal to generate a negative (unsat) instance
    
    # transforming each sat instance into tuple eases the generation of negative instances
    sat_list = [tuple(m) for m in solver.enum_models()] 
    #print(sat_list[0])
    sat_set = set(sat_list) # much quicker to verify an existing instance
    
    for assignment in sat_list:
        for i, literal in enumerate(assignment):
            tentative_unsat = list(assignment)
            tentative_unsat[i] = -tentative_unsat[i] #negating one literal
            if tuple(tentative_unsat) not in sat_set:
                unsat_list.append(tentative_unsat)
                break # goes on to next assignment
            #print(f'negated {i}-th')
    
    #for num, m in enumerate(solver.enum_models()):  
    #print(num)
print(len(sat_list))
print(len(unsat_list))

106
106
-1
1
